Hemos optado por sólo codificar en inglés, pero las explicaciones y comentarios del código estarán en español.

In [ ]:
# Importamos todas las librerías a utilizar:
import pandas as pd
import geopandas as gpd
import requests
import time
from tqdm import tqdm # Para barras de progreso, ayuda mucho ya que la API tiene delay de 1s por cada request!!
from shapely.geometry import Point
from shapely import wkt # Lo usaremos para parsear algunas columnas de algunos datasets a geometry, en un GDF.
import geemap,ee
import json

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-aesmatias')
#Aqui, por google colab no tuve que usar un token ni nada, pero se requiere autenticacion

Descargamos el Shapefile de USA desde https://gadm.org/download_country.html, eligiendo United States y descomprimiendo el .zip, eso nos dará los archivos necesarios, luego cargamos el Shapefile de USA y filtramos el AOI en Manhattan, para finalmente de transformarlo a GEOJSON y poder utilizarlo en GEE:

In [ ]:
# Nivel 2 para elegir los condados, luego lo pasamos a EPSG:4326, compatible con GEE
gdf_USA = gpd.read_file("gadm41_USA_2.shp").to_crs(epsg=4326)

ny_TO_GDF = gdf_USA[gdf_USA['NAME_2'] == 'New York'] # Seleccionamos New York
ny_TO_GDF.to_file("manhattan.geojson", driver="GeoJSON") # Hacemos un .geojson y lo guardamos

In [ ]:
# Cargamos el geojson creado, para que GEE lo pueda utilizar
gdf = gpd.read_file("manhattan.geojson")
manhattan_geojson = json.loads(gdf.to_json())
manhattan_ee = ee.FeatureCollection(manhattan_geojson)

Visualizamos manhattan con GEE, podemos ajustar los parámetros como la opacidad del AOI en el mapa interactivo:

In [ ]:
manhattanCollection = (ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(manhattan_ee)
    .filterDate("2024-05-01", "2025-04-01") # Mayo 2024 - Abril 2025
    .median() # Usamos la mediana de las imágenes de momento, sólo queremos apreciar el mapa
    .clip(manhattan_ee)) # Recortamos en Manhattan, la AOI

Map = geemap.Map(center=[40.783, -73.971], zoom=12)

Map.addLayer(manhattanCollection, {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000
}, 'RGB')

Map.addLayer(manhattan_ee.style(**{
    'width': 1,
    'color': 'red', # El borde será CYAN
    #'fillColor': '00000000',  # Color transparente de relleno
}), {}, 'AOI Manhattan')
Map

Se ha utilizado el dataset de Manhattan, obtenido en https://www.nyc.gov/site/finance/property/property-rolling-sales-data.page, la fecha de los registros del dataset está entre Mayo 2024 - Abril 2025.

Aquí, en las celdas iniciales que siguen, se muestra como se han geocodificado algunos datos con ayuda de una API, pero no hace falta usarlo, porque los datos procesados ya han sido guardados en un .csv (rollingsales_manhattan_geocoded.csv)

In [ ]:
'''La API que usamos para geocodificar en su capa gratuita es para testing, este proyecto
Universitario no califica como un proyecto de producción, no hay usuario final que lo utilizará.
Además, sólo hicimos uso de la API en pocas ocasiones'''

OPENCAGE_API_KEY = 'KEY'
OPENCAGE_BASE_URL = 'https://api.opencagedata.com/geocode/v1/json'

input_excel_file = './rollingsales_manhattan.xlsx' # XLSX (excel)

# Output y failed logs, para poder tener una reanudación cada 2500 request en la geocodificación
output_csv_file = './rollingsales_manhattan_geocoded.csv'
failed_addresses_file = './rollingsales_manhattan_geocoding_failures.csv'

MAX_DAILY_REQUESTS = 2500
REQUEST_DELAY = 1.0 # seconds

REQUIRED_COLUMNS = ['ADDRESS', 'BOROUGH', 'ZIP CODE', 'SALE PRICE']

Creamos la función que procesará cada solicitud a la API para poder geocodificar las direcciones y los ZIP codes en coordenadas geográficas que usaremos en los GDF posteriormente:

In [ ]:
# Esta función hace un llamado a la API, y retorna un array con la latitud, longitud y estado
def geocode_address(address_str, api_key, borough=None, zip_code=None):
    query = f"{address_str}, {borough}, New York, NY {zip_code}" if borough and zip_code else address_str

    params = {
        'q': query,
        'key': api_key,
        'language': 'en',
        'no_annotations': 1,
        'limit': 1
    }

    try:
        response = requests.get(OPENCAGE_BASE_URL, params=params)
        response.raise_for_status()
        data = response.json()

        if data and data['results']:
            lat = data['results'][0]['geometry']['lat']
            lng = data['results'][0]['geometry']['lng']

            components = data['results'][0].get('components', {})
            is_nyc = False # Empieza como false, y si lo encontramos, lo cambiamos a True:
            if 'state_code' in components and components['state_code'] == 'NY':
                if 'city' in components and components['city'] in ['New York', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island', 'Manhattan']:
                    is_nyc = True
                elif 'county' in components and ('New York County' in components['county'] or \
                                                 'Kings County' in components['county'] or \
                                                 'Queens County' in components['county'] or \
                                                 'Bronx County' in components['county'] or \
                                                 'Richmond County' in components['county']):
                    is_nyc = True

            if is_nyc:
                return lat, lng, "Success"
            else:
                return None, None, "Not_NYC_Result"
        else:
            return None, None, "No_Results"
    except requests.exceptions.RequestException as e:
        if response.status_code == 429:
            return None, None, "Rate_Limit_Exceeded"
        elif response.status_code == 401:
            print(f"API Key Error!!")
            return None, None, "API_Key_Error"
        elif response.status_code == 402: # Entonces, llegamos al límite de la quota diaria gratis
            print(f"ERROR, Quota exceded!")
            return None, None, "Payment_Required_Error"
        else:
            print(f"Request error: {e}")
            return None, None, f"Request_Error: {e}"
    except Exception as e:
        print(f"ERROR: {e}")
        return None, None, f"Error: {e}"

Comenzamos a procesar las direcciones y ZIP codes a coordenadas geográficas:

In [ ]:
print(f"*** Procesando {input_excel_file} ***")

df = None
# Agregamos las columnas requeridas que fallaron a entradas del df fallido, para re intentar si se quisiera:
failed_df = pd.DataFrame(columns=REQUIRED_COLUMNS + ['Reason'])

try:
    if pd.io.common.file_exists(output_csv_file):
        print(f"Fichero '{output_csv_file}' con progreso encontrado - Resumiendo...")
        df = pd.read_csv(output_csv_file)
        # Si no hay latitud, longitud o estado de la geocodificación, sabemos que la entrada no ha sido procesada:
        for col in ['LATITUDE', 'LONGITUDE', 'GEOCODING_STATUS']:
            if col not in df.columns:
                df[col] = None
            df[col] = df[col].astype(object)

    else:
        print(f"No se encontró el fichero con progreso, cargando el fichero inicial XLSX: '{input_excel_file}'.")
        found_header = False
        # Probamos con headers desde el 0 al 10, porque algunos datasets XLSX tienen las primeras entradas con información,
        # hay que evitar las primeras entradas que no son los headers, para no obtener errores:
        for header_row_index in range(10):
            try:
                print(f"Intentando cargar con el header {header_row_index}")
                df_temp = pd.read_excel(input_excel_file, header=header_row_index)

                # Check if all required columns are present in the loaded DataFrame
                if all(col in df_temp.columns for col in REQUIRED_COLUMNS):
                    df = df_temp
                    found_header = True
                    print(f"Header encontrado en el índice {header_row_index}!")
                    print("Columnass encontradas en el DF:", df.columns.tolist())
                    break # Si el header es encontrado, dejamos de loopear
                else:
                    print(f"No hay header en el índice {header_row_index}, probando el siguiente...")
            except Exception as e:
                print(f"Error: {e}")

        if not found_header: # Si no hay header, hay un error en el fichero
            print(f"No se ha encontrado un header, error en el fichero XLSX")
            exit()

        # Limpiamos las columnas del DF y las parseamos
        df['ADDRESS'] = df['ADDRESS'].fillna('').astype(str)
        df['NEIGHBORHOOD'] = df['NEIGHBORHOOD'].fillna('').astype(str)
        df['BOROUGH'] = df['BOROUGH'].fillna('').astype(str)
        df['ZIP CODE'] = df['ZIP CODE'].fillna(0).astype(int).astype(str).replace('0', '')

        # Agregamos la latitud, longitud, y estado de la geocodificación, como nuevas columnas en el DF:
        df['LATITUDE'] = None
        df['LONGITUDE'] = None
        df['GEOCODING_STATUS'] = None

    already_geocoded_count = df['LATITUDE'].notna().sum()
    requests_made_now = 0
    print(f"Los registros geocodificados hasta el momento son: {already_geocoded_count}")

    # La siguiente línea veririfica si la latitud está vacía y el "status" es diferente de "Not_NYC_Result"
    # Si el registro tiene GEOCODIG_STATUS = 'Not_NYC_Result', entonces ha fallado la geocodificación.
    rows_to_geocode = df[(df['LATITUDE'].isna()) & (df['GEOCODING_STATUS'] != 'Not_NYC_Result')]

    print(f"Han fallado: {len(rows_to_geocode)} registros")

    for index, row in tqdm(rows_to_geocode.iterrows(), total=len(rows_to_geocode), desc="Geocoding"): #tqdm para barra de progrso
        if requests_made_now >= MAX_DAILY_REQUESTS:
            print(f"Se ha llegado al límite de {MAX_DAILY_REQUESTS} requests diarias!")
            break

        address = row['ADDRESS']
        borough = row['BOROUGH']
        zip_code = row['ZIP CODE'] if row['ZIP CODE'] != '0' else ''

        if not address:
            df.loc[index, 'GEOCODING_STATUS'] = "Empty_Address"
            continue

        lat, lon, status = geocode_address(address, OPENCAGE_API_KEY, borough, zip_code)
        requests_made_now += 1

        df.loc[index, 'LATITUDE'] = lat
        df.loc[index, 'LONGITUDE'] = lon
        df.loc[index, 'GEOCODING_STATUS'] = status

        if status in ["Rate_Limit_Exceeded", "API_Key_Error", "Payment_Required_Error"]:
            break

        time.sleep(REQUEST_DELAY) # La documentación de la API indica un delay de 1 segundo entre cada request

        # Guardamos el progreso en chunks de cada 100 solicitudes a la API:
        if requests_made_now % 100 == 0:
            print(f"Guardando progreso en la request número {requests_made_now}")
            df.to_csv(output_csv_file, index=False)

    df.to_csv(output_csv_file, index=False)
    print(f"Geocodificación finalizada!!")

    failed_rows = df[df['LATITUDE'].isna()] # Si no tiene latitud, es una row fallida
    if not failed_rows.empty:
        failed_df_to_save = failed_rows[['BOROUGH', 'NEIGHBORHOOD', 'ADDRESS', 'ZIP CODE', 'GEOCODING_STATUS']].copy()
        # Guardamos en failed_addresses_file sólo las rows de la variable de arriba, para poder procesarlas luego.
        failed_df_to_save.to_csv(failed_addresses_file, index=False)
        print(f"Los registros fallidos se guardaron en: {failed_addresses_file}")
    else:
        print("Atención! Finalización inesperada, posiblemente ha ocurrido un error.")

except FileNotFoundError:
    print(f"ERROR: Archivo '{input_excel_file}' no hallado.")
except Exception as e:
    print(f"ERROR: {e}")

Cargamos el fichero con la latitud y longitud agregadas en la geocodificación, para luego, filtrar los registros que tengan NaN en latitud y longitud, ya que eso es producto de errores en la geocodificación de dichos valores. También cribamos y sólo tomamos como válidos valores con precio de venta mayor a 0, ya que hay varios valores en el dataset con propiedades que se han vendido a costo 0, lo que no tiene representación estadística en nuestro contexto.

In [ ]:
properties_geocoded_file = './rollingsales_manhattan_geocoded.csv'

try:
    df_properties = pd.read_csv(properties_geocoded_file)
    print(f"***Cargando fichero: {properties_geocoded_file}*** \n")
    print(f"Cantidad de registros encontrados en {str(properties_geocoded_file)}: {len(df_properties)} \n")
    print("df_properties.info(): \n")
    print(df_properties.info())

except Exception as e:
    print(f"Error: {str(e)}")
    exit()

# Eliminamos los registros que sean NaN en latitud y longitud.
df_properties.dropna(subset=['LATITUDE', 'LONGITUDE'], inplace=True)
print(f"Registros después de eliminar NaN en lat/lon: {len(df_properties)}")

#Eliminamos los registros que tengan precio de venta menor o igual a 0, sin representación estadística.
df_properties = df_properties[df_properties['SALE PRICE'] > 0]
print(f"Registros después de filtrar por precio de venta > 0: {len(df_properties)}")

Luego, para una posterior manipulación y tener una mayor compatibilidad con GEE y librerías para graficar, "transformamos" el dataframe a un geodataframe, y le agregamos una nueva columna de tipo geometry, que contendrá puntos generados a través de las propiedades de latitud y longitud, los cuales están definidos en la variable geometry, haciendo uso del métetodo zip y list comprehension.

In [ ]:
# A partir de las coordenadas, creamos objetos de tipo Point, de lalibrería shapely.geometry, para luego manipular mejor:
geometry = [Point(xy) for xy in zip(df_properties['LONGITUDE'], df_properties['LATITUDE'])]

# Creamos el nuevo geodataframe para, a partir del dataframe anterior, llenarlo con los datos:
gdf_properties = gpd.GeoDataFrame(df_properties, geometry=geometry, crs="EPSG:4326") #CRS es EPSG:4326 para latitud y longitud.

print("Nuevo GeoDataFrame:")
print(gdf_properties.info())

Cargamos el dataset https://catalog.data.gov/dataset/nypd-shooting-incident-data-historic que contiene el histórico de tiroteos.

In [ ]:
shooting_incident_historic = 'NYPD_Shooting_Incident_Data__Historic_.csv'
df_shooting_incident_historic = None

try:
    df_shooting_incident_historic = pd.read_csv(shooting_incident_historic)
    print(f"El archivo {shooting_incident_historic} ha sido cargado\n Longitud: {len(df_shooting_incident_historic)} \n")
    print(f'Información del fichero: \n')
    df_shooting_incident_historic.info()
except Exception as e:
    print(f"Error: {e}")

 Al igual que antes, este dataset también debe ser filtrado por fecha de interés y ser convertido a GDF, la columna con el Point, que contiene la geometría de la latitud y longitud, está en una columna llamada "Lon_Lat", y contiene valores de tipo Point.

In [ ]:
LATITUDE_COL = 'Latitude'
LONGITUDE_COL = 'Longitude'
INCIDENT_DATE_COL = 'OCCUR_DATE'
BORO_COL = 'BORO'
year_of_preference = 2024 # Nos interesan datos de tiroteos en los últimos 2 años

# Cargamos el archivo y definimos una variable para su dataframe
shooting_incident_historic = 'NYPD_Shooting_Incident_Data__Historic_.csv'

try:
    df_shooting_incident_historic = pd.read_csv(shooting_incident_historic)
    print(f"El archivo {shooting_incident_historic} ha sido cargado. Su longitud es: {len(df_shooting_incident_historic)} \n")
    df_shooting_incident_historic.info()
except Exception as e:
    print(f"Error: {e}")

# Eliminamos las filas con NaN en las coordenadas
df_shooting_incident_historic.dropna(subset=[LATITUDE_COL, LONGITUDE_COL], inplace=True)

# Si la columna con fecha del incidente existe, filtramos por BORO y AÑO:
if INCIDENT_DATE_COL in df_shooting_incident_historic.columns:
    # Convertimos la columna de OCCUR_DATE a datatime de pandas, para trabajarla con el formato de USA:
    df_shooting_incident_historic[INCIDENT_DATE_COL] = pd.to_datetime(
        df_shooting_incident_historic[INCIDENT_DATE_COL],
        format='%m/%d/%Y',
        errors='raise' # Podemos poner coerce
    )
    df_shooting_incident_historic.dropna(subset=[INCIDENT_DATE_COL], inplace=True)   # Dropeamos valores sin fecha válida.
    df_shooting_incident_historic['INCIDENT_YEAR'] = df_shooting_incident_historic[INCIDENT_DATE_COL].dt.year

    # Filtramos para sólo obtener datos de Manhattan (en la col BORO)
    if BORO_COL in df_shooting_incident_historic.columns:
        # Usamos .copy() para definir la variable por valor, y no por referencia en memoria:
        df_shooting_incident_manhattan = df_shooting_incident_historic[df_shooting_incident_historic[BORO_COL] == 'MANHATTAN'].copy()
        print(f"Encontramos: {len(df_shooting_incident_manhattan)} incidentes en Manhattan")

        # Filtramos según el año deseado:
        df_shooting_incident_manhattan = df_shooting_incident_manhattan[df_shooting_incident_manhattan['INCIDENT_YEAR'] >= year_of_preference]
        print(f"Encontramos: {len(df_shooting_incident_manhattan)} incidentes posteriores al año {year_of_preference}. \n")

        # Antes de convertir el DF a GDF, necesitamos col geometry que contiene puntos, los cuales están en la
        # columna Lon_Lat, por lo que la parseamos:
        df_shooting_incident_manhattan['Lon_Lat'] = df_shooting_incident_manhattan['Lon_Lat'].apply(wkt.loads)

        # Creamos el GeoDataFrame de los incidentes y lo asignamos en una nueva variable:
        geometry_incidents = [Point(xy) for xy in zip(df_shooting_incident_manhattan[LONGITUDE_COL], df_shooting_incident_manhattan[LATITUDE_COL])]


        # Renombramos la columna Lon_Lat a geometry, ya que sabemos que existe Lon_Lat, que es un tipo de dato Point
        df_shooting_incident_manhattan.rename(columns={'Lon_Lat': 'geometry'}, inplace=True)

        # Creamos el GDF utilizando la col geometry:
        gdf_incidents = gpd.GeoDataFrame(df_shooting_incident_manhattan, geometry='geometry', crs="EPSG:4326")  # CRS 4326 para lat/lon

        print("Información del GDF:")
        gdf_incidents.info()
    else:
      raise KeyError(f"Error: La columna '{BORO_COL}' no se encontró!! ")